# Insert

The `insert` operation adds new entities to Manual tables.
In the context of the [Relational Workflow Model](../20-concepts/05-workflows.md), inserting data is how information enters the pipeline from external sources.

## Insert in the Workflow

The `insert` operation applies to **Manual tables**—tables that receive data from outside the pipeline:

| Table Tier | How Data Enters |
|------------|-----------------|
| **Lookup** | `contents` property (part of schema definition) |
| **Manual** | `insert` from external sources |
| **Imported/Computed** | `populate()` mechanism |

For **Manual tables**, each insert represents new information entering the workflow from an external source.
The term "manual" refers to the data's origin—*outside the pipeline*—not to how it arrives.
Inserts into Manual tables can come from human data entry, automated scripts parsing instrument files, or integrations with external systems.
What matters is that the pipeline's `populate` mechanism does not create this data—it comes from outside.

Each insert into a Manual table potentially triggers downstream computations: when you insert a new session, all Imported and Computed tables that depend on it become candidates for population.

## The `insert1` Method

Use `insert1` to add a single row:

```python
<Table>.insert1(row, ignore_extra_fields=False)
```

**Parameters:**
- **`row`**: A dictionary with keys matching table attributes
- **`ignore_extra_fields`**: If `True`, extra dictionary keys are silently ignored; if `False` (default), extra keys raise an error

**Example:**
```python
# Insert a single subject into a Manual table
Subject.insert1({
    'subject_id': 'M001',
    'species': 'mouse',
    'sex': 'M',
    'date_of_birth': '2023-06-15'
})
```

Use `insert1` when:
- Adding individual records interactively
- Processing items one at a time in a loop where you need error handling per item
- Debugging, where single-row operations provide clearer error messages

## The `insert` Method

Use `insert` for batch insertion of multiple rows:

```python
<Table>.insert(rows, ignore_extra_fields=False, skip_duplicates=False)
```

**Parameters:**
- **`rows`**: A list of dictionaries (or any iterable of dict-like objects)
- **`ignore_extra_fields`**: If `True`, extra keys are ignored
- **`skip_duplicates`**: If `True`, rows with existing primary keys are silently skipped; if `False` (default), duplicates raise an error

**Example:**
```python
# Batch insert multiple sessions (could be from a script parsing log files)
Session.insert([
    {'subject_id': 'M001', 'session_date': '2024-01-15', 'session_notes': 'baseline'},
    {'subject_id': 'M001', 'session_date': '2024-01-16', 'session_notes': 'treatment'},
    {'subject_id': 'M001', 'session_date': '2024-01-17', 'session_notes': 'follow-up'},
])
```

Use `insert` when:
- Loading data from files or external sources
- Importing from external databases or APIs
- Migrating or synchronizing data between systems

## Referential Integrity

DataJoint enforces referential integrity on insert.
If a table has foreign key dependencies, the referenced entities must already exist:

```python
# This will fail if subject 'M001' doesn't exist in Subject table
Session.insert1({
    'subject_id': 'M001',  # Must exist in Subject
    'session_date': '2024-01-15'
})
```

This constraint ensures the dependency graph remains valid—you cannot create downstream entities without their upstream dependencies.
Note that Lookup table data (defined via `contents`) is automatically available when the schema is activated, so foreign key references to Lookup tables are always satisfied.

## Best Practices

1. **Match insert method to use case**: Use `insert1` for single records, `insert` for batches
2. **Keep `ignore_extra_fields=False`** (default): Helps catch data mapping errors early
3. **Insert upstream before downstream**: Respect the dependency order defined by foreign keys
4. **Use `skip_duplicates=True` for idempotent scripts**: When re-running import scripts, this avoids errors on existing data
5. **Let `populate()` handle auto-populated tables**: Never insert directly into Imported or Computed tables